In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [3]:
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

In [4]:
pcm.calculate_q(sys.atoms, 4)

In [5]:
sys.atoms.keys()

dict_keys(['positions', 'ids', 'types', 'ghost', 'neighbors', 'neighbor_dist', 'temp_neighbors', 'temp_neighbordist', 'neighborweight', 'diff', 'r', 'theta', 'phi', 'cutoff', 'neighbordist', 'q0', 'q0_real', 'q0_imag', 'q1', 'q1_real', 'q1_imag', 'q2', 'q2_real', 'q2_imag', 'q3', 'q3_real', 'q3_imag', 'q4', 'q4_real', 'q4_imag'])

In [6]:
sys.q4

[0.03636964837266554,
 0.036369648372665514,
 0.036369648372665514,
 0.03636964837266552,
 0.03636964837266552,
 0.03636964837266552,
 0.03636964837266554,
 0.03636964837266553,
 0.03636964837266552,
 0.03636964837266553,
 0.03636964837266549,
 0.03636964837266554,
 0.036369648372665514,
 0.03636964837266552,
 0.036369648372665514,
 0.036369648372665514,
 0.036369648372665514,
 0.036369648372665514,
 0.03636964837266551,
 0.036369648372665514,
 0.03636964837266553,
 0.036369648372665514,
 0.03636964837266552,
 0.03636964837266551,
 0.036369648372665514,
 0.03636964837266551,
 0.036369648372665535,
 0.036369648372665514,
 0.03636964837266552,
 0.036369648372665514,
 0.03636964837266548,
 0.03636964837266552,
 0.0363696483726655,
 0.03636964837266552,
 0.03636964837266552,
 0.03636964837266552,
 0.0363696483726655,
 0.03636964837266552,
 0.03636964837266551,
 0.03636964837266552,
 0.03636964837266551,
 0.03636964837266551,
 0.0363696483726655,
 0.03636964837266552,
 0.03636964837266551,
